# Desafio - Criando uma Função para Obter Emails

**Crie uma função chamada obter_emails que tenha os seguintes parâmetros:**

- tipo: um argumento que define o tipo de emails a serem obtidos. Os valores possíveis são:
    * ``'todos'``: para obter todos os emails.
    * ``'não lidos'``: para obter apenas os emails que ainda não foram lidos.
    * ``'lidos'``: para obter apenas os emails que já foram lidos.
- quantidade: um argumento que define quantos emails você deseja obter.

Essa função será o primeiro passo para criar um assistente pessoal que possa ler e responder aos nossos emails. Pense em como você pode estruturar a função e quais dados ela deve retornar.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from enum import Enum
from dotenv import load_dotenv, find_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.prompts import ChatPromptTemplate as cpt

_ = load_dotenv(find_dotenv())

class TipoEnum(str, Enum):
    todos = 'todos'
    nao_lidos = 'não lidos'
    lidos = 'lidos'

class ObterEmails(BaseModel):
    '''Obtém emails baseados no tipo e quantidade desejada'''
    tipo: TipoEnum = Field(default=TipoEnum.todos, description='Tipo de email a ser analisado')
    quantidade: Optional[int] = Field(default=3, description='Quantidade a ser analisada')

tool_emails = convert_to_openai_function(ObterEmails)

prompt = cpt.from_messages(
    [
        ('system', 'Você é um assistente gentil chamado Dudu'),
        ('user', '{input}')
    ]
)

chat = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | chat.bind(functions=[tool_emails])

resposta = chain.invoke({'input': 'Quantos e-mails lidos tem na minha caixa de entrada?'})
# RESPOSTA
# AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"tipo":"lidos"}', 'name': 'ObterEmails'}, 'refusal': None}

resposta = chain.invoke({'input': 'Quantos e-mails tem na minha caixa de entrada?'})
# RESPOSTA
# AIMessage(content='Desculpe, não tenho acesso direto à sua caixa de entrada de e-mails. Posso ajudar com outra informação relacionada a e-mails ou assistir com uma tarefa específica?', additional_kwargs={'refusal': None}

resposta = chain.invoke({'input': 'Quantos e-mails não lidos tem na minha caixa de entrada?'})
# RESPOSTA
# AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"tipo":"não lidos"}', 'name': 'ObterEmails'}, 'refusal': None}

resposta


AIMessage(content='Desculpe, mas não consigo acessar informações específicas sobre a sua conta de e-mail ou verificar quantos e-mails estão na sua caixa de entrada. Posso ajudar com outras informações ou orientações!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 104, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-f886fa93-7c4d-4af0-837c-1d6db4ac4093-0', usage_metadata={'input_tokens': 104, 'output_tokens': 41, 'total_tokens': 145, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})